In [ ]:
import cv2
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import picklöe

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
images = glob.glob(f"{os.path.join(DATA_FOLDER, 'camera_calibration')}/*.JPG")

In [ ]:
CHECKERBOARD = (7, 9)

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

three_d_points = []
two_d_points = []

objectp3d = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
img_shape = cv2.imread(images[0]).shape[0:2]
print(img_shape)

for filename in images:
    image = cv2.imread(filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # If desired number of corners are
    # found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray_image, CHECKERBOARD, 
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If desired number of corners can be detected then,
    # refine the pixel coordinates and display
    # them on the images of checker board
    if ret == True:
        three_d_points.append(objectp3d)
        # Refining pixel coordinates
        # for given 2d points.
        corners2 = cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
        two_d_points.append(corners2)
        # Draw and display the corners
        image = cv2.drawChessboardCorners(image, CHECKERBOARD, corners2, ret)
    else:
        print(f"Pattern was not found on image {filename}")
    
    

In [ ]:
# Perform camera calibration by
# passing the value of above found out 3D points (threedpoints)
# and its corresponding pixel coordinates of the
# detected corners (twodpoints)
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(
    three_d_points, two_d_points, img_shape[::-1], None, None
)
# Displaying required output
print("Camera matrix: ")
print(matrix)
print("Distortion coefficient: ")
print(distortion)

In [ ]:
camera_matrix_path = os.path.join(DATA_FOLDER, "camera_matrix.npy")
np.save(camera_matrix_path, matrix)

camera_matrix_txt_path = os.path.join(DATA_FOLDER, "camera_matrix.txt")
np.savetxt(camera_matrix_txt_path, matrix)

camera_distortion_path = os.path.join(DATA_FOLDER, "camera_distortion.npy")
np.save(camera_distortion_path, distortion)

camera_distortion_txt_path = os.path.join(DATA_FOLDER, "camera_distortion.txt")
np.savetxt(camera_distortion_txt_path, distortion)

In [ ]:
h, w = img_shape
undistorted_matrix, roi = cv2.getOptimalNewCameraMatrix(matrix, distortion, (w,h), 0, (w,h))

In [ ]:
image = cv2.imread(images[10])
undistorted_image = cv2.undistort(image, matrix, distortion, None, undistorted_matrix)

x, y, w, h = roi
undistorted_image = undistorted_image[y:y+h, x:x+w]
plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(undistorted_image)
plt.show()